## Criação da partições de competition

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
import boto3

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df_competition = spark.read.option("multiline", "true").json("s3://sor/football/data/competitions.json")

24/01/05 04:26:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [79]:
df_competition.show()
df_competition.count()

+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|competition_gender|competition_id|competition_international|competition_name|competition_youth|country_name|     match_available|match_available_360|       match_updated|   match_updated_360|season_id|season_name|
+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|              male|             9|                    false|   1. Bundesliga|            false|     Germany|2023-12-12T07:43:...|               null|2023-12-12T07:43:...|                null|       27|  2015/2016|
|              male|            16|                    false|Champions League|            false|      Europe|2023-03-07T12:20:...|          

70

In [70]:
# Selecionar temporadas distintas
df_competition.select("season_id").distinct().orderBy("season_id").count()

45

In [50]:
partitions = df_competition.select('season_id').distinct().orderBy("season_id").collect()
print(partitions)

[Row(season_id=1), Row(season_id=2), Row(season_id=3), Row(season_id=4), Row(season_id=21), Row(season_id=22), Row(season_id=23), Row(season_id=24), Row(season_id=25), Row(season_id=26), Row(season_id=27), Row(season_id=30), Row(season_id=37), Row(season_id=38), Row(season_id=39), Row(season_id=40), Row(season_id=41), Row(season_id=42), Row(season_id=43), Row(season_id=44), Row(season_id=48), Row(season_id=51), Row(season_id=54), Row(season_id=55), Row(season_id=68), Row(season_id=71), Row(season_id=75), Row(season_id=76), Row(season_id=84), Row(season_id=86), Row(season_id=90), Row(season_id=106), Row(season_id=107), Row(season_id=108), Row(season_id=235), Row(season_id=268), Row(season_id=269), Row(season_id=270), Row(season_id=272), Row(season_id=274), Row(season_id=275), Row(season_id=276), Row(season_id=277), Row(season_id=278), Row(season_id=279)]


In [62]:
dataframes = {}
for partition in partitions:
    partition_value = partition['season_id']
    partition_competition = df_competition.filter(df_competition['season_id'] == partition_value)
    dataframes[partition_value] = partition_competition

In [73]:
# Caminho base no S3
base_path = "s3://landing/"

# Iterar sobre os DataFrames e salvá-los no S3
for partition_value, dataframe in dataframes.items():
    path = f"{base_path}/competitions/season={partition_value}"
    dataframe.write.json(path, mode='overwrite')

In [78]:
read_df = spark.read.json(f"{base_path}/competitions")
read_df.show()
read_df.count()

+------------------+--------------+-------------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+
|competition_gender|competition_id|competition_international|    competition_name|competition_youth|        country_name|     match_available| match_available_360|       match_updated|   match_updated_360|season_id|season_name|season|
+------------------+--------------+-------------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+
|              male|             9|                    false|       1. Bundesliga|            false|             Germany|2023-12-12T07:43:...|                null|2023-12-12T07:43:...|                null|       27|  2015/2016|    27|
|              male|            16|                    false

70

## Criação das partições de Matches

In [5]:
df_matches_competition27 = spark.read.option("multiline", "true").json("s3://sor/football/data/matches/2/27.json")
df_matches_competition27.show()

+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+
|away_score|           away_team|         competition|  competition_stage|home_score|           home_team|    kick_off|        last_updated|    last_updated_360|match_date|match_id|match_status|match_status_360|match_week|     metadata|             referee|         season|             stadium|
+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+
|         0|{male, null, 28, ...|{2, Premier Leagu...|{1, Regular Season}|         0|{{68, England}, m...|16:00:00.

In [21]:
# Mapear todas as competições dos jogos

bucket_path = "s3://sor/football/data/matches/"

# Lê dados do bucket usando um DataFrame Spark
df_conteudo_bucket = spark.read.format("s3").option("recursive", "true").load(bucket_path)

df_conteudo_bucket.show()




Py4JJavaError: An error occurred while calling o142.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: s3. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:207)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:185)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(Me

In [6]:
competition_id = "2"

colunas_particao = [competition_id, "season.season_id"]

In [7]:
base_path = "s3://landing/"

df_matches_competition27.write \
    .partitionBy(*colunas_particao) \
    .json(f"{base_path}/matches/{competition_id}/27.json")

AnalysisException: Partition column `2` not found in schema struct<away_score:bigint,away_team:struct<away_team_gender:string,away_team_group:string,away_team_id:bigint,away_team_name:string,country:struct<id:bigint,name:string>,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>,competition:struct<competition_id:bigint,competition_name:string,country_name:string>,competition_stage:struct<id:bigint,name:string>,home_score:bigint,home_team:struct<country:struct<id:bigint,name:string>,home_team_gender:string,home_team_group:string,home_team_id:bigint,home_team_name:string,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>,kick_off:string,last_updated:string,last_updated_360:string,match_date:string,match_id:bigint,match_status:string,match_status_360:string,match_week:bigint,metadata:struct<data_version:string,shot_fidelity_version:string,xy_fidelity_version:string>,re

In [ ]:
# Percorrer todos os jogos de todas as competições